In [ ]:
import sys
import json
import pandas as pd
import numpy as np
from tqdm import tqdm  # Used to measure progress of long a running script.

In [ ]:
pd.set_option('display.max_columns', None)  # Show all columns when displaying DataFrames.

In [ ]:
drop_list = ['tpk-niputuspvm',
             'ohjaaja_ytunnus_kj_tutkinto',
             'opiskeluoikeus_oid']

rename_dict = {'tyopaikan_normalisoitu_nimi': 'tyopaikka_normalisoitu',
               'tyopaikan_nimi': 'tyopaikka',
               'oppilaitos': 'oppilaitos_oid',
               'koulutustoimija': 'koulutustoimija_oid',
               'alkupvm':'vastaamisajan_alkupvm',
               'viimeinen_vastauspvm': 'vastaamisajan_loppupvm',
               'oppilaitos': 'oppilaitos_oid',
               'uusi-kesto-without-oa': 'tyopaikkajakson_kesto',
               'hankkimistapa_tyyppi': 'sopimustyyppi',
               'tyopaikan_ytunnus': 'tyonantaja',
               'tutkinto': 'tutkintotunnus',
               'tutkinnonosa_nimi': 'paikallinen_tutkinnon_osa',
               'jakso_loppupvm': 'tyopaikkajakson_loppupvm',
               'jakso_alkupvm': 'tyopaikkajakson_alkupvm',
               'tutkinnonosa_koodi': 'tutkinnon_osa',
               'existing-arvo-tunnus': 'tunnus'}

types = {
    "osa_aikaisuus": 'string',
    "tyopaikka": 'string',
    "rahoitusryhma": 'string',
    "tyopaikka_normalisoitu": 'string',
    "vastaamisajan_alkupvm": 'string',
    "tyonantaja": 'string',
    "oppisopimuksen_perusta": 'string',
    "tyopaikkajakson_kesto": 'string',
    "osaamisala": 'string',
    "koulutustoimija_oid": 'string',
    "paikallinen_tutkinnon_osa": 'string',
    "tyopaikkajakson_loppupvm": 'string',
    "toimipiste_oid": 'string',
    "oppilaitos_oid": 'string',
    "tyopaikkajakson_alkupvm": 'string',
    "tutkintotunnus": 'string',
    "sopimustyyppi": 'string',
    "tutkintonimike": 'string',
    "request_id": 'string',
    "vastaamisajan_loppupvm": 'string',
    "tutkinnon_osa": 'string'
}

In [ ]:
with open('jaksotunnus-taulun-data.json') as file:
    data = json.load(file)
    vanha_df = pd.DataFrame(data)
    vanha_df = vanha_df.set_index('hankkimistapa_id')
    vanha_df = vanha_df.drop(columns=drop_list)
    vanha_df = vanha_df.rename(columns=rename_dict)

with open('rahoituslaskenta-taulun-data.json') as file:
    data = json.load(file)
    uusi_df = pd.DataFrame(data)
    uusi_df = uusi_df.set_index('hankkimistapa_id')
    uusi_df = uusi_df.drop(columns=drop_list)
    uusi_df = uusi_df.drop(columns=['vanha-kesto', 'uusi-kesto-with-oa', 'save-timestamp', 'tallennuspvm'])
    uusi_df = uusi_df.fillna('')
    uusi_df = uusi_df.rename(columns=rename_dict)

In [ ]:
# Suodatetaan jaksot joissa kesto yli 1500 päivää.
uusi_df = uusi_df[uusi_df.tyopaikkajakson_kesto <= 1500]

In [ ]:
def get_tutkinnonosa(x):
    splitted = x.split('_')
    if len(splitted) == 2:
        return splitted[1]
    return x

def to_comma_separated_list(x):
    # Yksinkertaiseksi listaksi, jossa elementit eroteltu pilkulla.
    return ','.join(map(lambda y: y[1:-1], x[1:-1].split()))
    # Sama kuin yllä, paitsi ympäröity lisäksi hakasulkeilla.
    # return ','.join(x.replace('(', '[').replace(')', ']').split())

In [ ]:
# Ensimmäinen datasetti: Jaksot joihin löytynyt match hankkimistapa-id:n perusteella.
jaksot_tunnuksilla_1 = uusi_df
jaksot_tunnuksilla_1 = jaksot_tunnuksilla_1.astype(types)

jaksot_tunnuksilla_1['tutkinnon_osa'] = jaksot_tunnuksilla_1['tutkinnon_osa'].apply(get_tutkinnonosa)
jaksot_tunnuksilla_1['tutkintonimike'] = jaksot_tunnuksilla_1['tutkintonimike'].apply(to_comma_separated_list)
jaksot_tunnuksilla_1['osaamisala'] = jaksot_tunnuksilla_1['osaamisala'].apply(to_comma_separated_list)

jaksot_tunnuksilla_1 = jaksot_tunnuksilla_1[jaksot_tunnuksilla_1['tunnus'] != '']
jaksot_tunnuksilla_1 = jaksot_tunnuksilla_1.drop(columns=['niputuspvm', 'oppija_oid','hoks_id', 'rahoituskausi', 'ohjaaja_nimi','ohjaaja_email', 'ohjaaja_puhelinnumero', 'tutkinnonosa_id', 'tutkinnonosa_tyyppi'])
jaksot_tunnuksilla_1 = jaksot_tunnuksilla_1.set_index('tunnus')
jaksot_tunnuksilla_1.to_csv('jaksot-tunnuksilla-1.2.csv', sep='|')
jaksot_tunnuksilla_1

In [ ]:
uusi_df[uusi_df.tunnus != '']

In [ ]:
def get_tutkinnonosa(x):
    splitted = x.split('_')
    if len(splitted) == 2:
        return splitted[1]
    return x

uusi_df['tutkinnon_osa'].apply(get_tutkinnonosa)

In [ ]:
vanha_df[(vanha_df.tyopaikkajakson_loppupvm < "2022-07-01") & (vanha_df.tunnus != '')]
# vanha_df

In [ ]:
vanha_df[~vanha_df.tunnus.isin(uusi_df.tunnus[uusi_df.tunnus != ''])]

In [ ]:
# tepRahoitusTablen jaksot joille ei ole vielä löydetty Arvo-tunnusta hankkimistapa-id:n perusteella.
uusi_without_tunnus = uusi_df[uusi_df.tunnus == '']
# Kopio yllä olevasta DataFramesta. Tätä täytetään alla olevassa silmukassa.
tunnukset_found = uusi_without_tunnus
# Jaksot jaksotunnus-taulusta, joista on poistettu ne jaksot jotka on jo mätsätty hankkimistapa-id:n perusteella.
vanha_unmatched = vanha_df[~vanha_df.tunnus.isin(uusi_df.tunnus[uusi_df.tunnus != ''])]
# Suodatetaan pois uuden rahoituskauden jaksot.
vanha_unmatched = vanha_unmatched[vanha_unmatched.tyopaikkajakson_loppupvm < "2022-07-01"]
# Uniikit HOKS ID:t listaan.
unique_hoks_ids = np.unique(uusi_df['hoks_id'])

for hoks_id in tqdm(unique_hoks_ids):
    
    # Haetaan hoks_id:tä vastaavat rivit (jaksot) jaksotunnus-taulun datasta
    # (`vanha_df`) ja tepRahoitusLaskenta-taulun datasta (`uusi_df`).
    rows_uusi = uusi_without_tunnus[uusi_without_tunnus.hoks_id == hoks_id]
    rows_vanha = vanha_unmatched[vanha_unmatched.hoks_id == hoks_id]
    
    ensisijaiset_kentat = ['tyonantaja', 'tutkinnon_osa']
    
    toissijaiset_kentat = ['tyopaikkajakson_loppupvm',
                           'tyopaikkajakson_alkupvm',
                           'ohjaaja_email',
                           'ohjaaja_nimi',
                           'ohjaaja_puhelinnumero',
                           'sopimustyyppi',
                           'tutkinnonosa_tyyppi']
    toissijaiset_kentat_reversed = toissijaiset_kentat.copy()
    toissijaiset_kentat_reversed.reverse()
    toissijaiset_kentat_2 = ['ohjaaja_email',
                             'ohjaaja_puhelinnumero',
                             'ohjaaja_nimi',
                             'tyopaikkajakson_alkupvm',
                             'tyopaikkajakson_loppupvm',
                             'tutkinnonosa_tyyppi',
                             'sopimustyyppi']
    
    ts_kentat_variaatiot = [toissijaiset_kentat,
                            toissijaiset_kentat_reversed,
                            toissijaiset_kentat_2]
        
    for index,row in rows_uusi.iterrows():
        
        match_loytynyt = False
        
        for ts_kentat in ts_kentat_variaatiot:
            
            rows_trimmed = rows_vanha
            
            for kentta in ensisijaiset_kentat + ts_kentat:
                rows_trimmed = rows_trimmed[rows_trimmed[kentta] == row[kentta]]

                # * Jos jäljelle jääneitä rivejä on vain yksi, voidaan olla varmoja että
                #   rivin `row` jakso vastaa jäljelle jääneen rivin jaksoa.
                if rows_trimmed.shape[0] == 1:
                    tunnukset_found.at[index, 'tunnus'] = rows_trimmed.tunnus.values[0]
                    match_loytynyt = True
                    break

            if match_loytynyt:
                break
        
        # if not match_loytynyt:
        #     tunnukset_found.at[index, 'tunnus'] = ','.join(rows_trimmed.tunnus.values)

In [ ]:
tunnukset_found_old = tunnukset_found

In [ ]:
tunnukset_found[tunnukset_found.tunnus != '']

In [ ]:
tunnukset_found_cat = pd.concat([tunnukset_found[tunnukset_found.tunnus != ''], uusi_df[uusi_df.tunnus != '']]).dropna(subset=['tunnus'])

In [ ]:
# Toinen datasetti: Jaksot joihin löytynyt match hankkimistapa-id:n perusteella tai joihin tunnus on muuten pystytty päättelemään.
jaksot_tunnuksilla_2 = tunnukset_found_cat[tunnukset_found_cat.tunnus != '']
jaksot_tunnuksilla_2 = jaksot_tunnuksilla_2.astype(types)
jaksot_tunnuksilla_2 = jaksot_tunnuksilla_2[~jaksot_tunnuksilla_2.index.isin([...])]

jaksot_tunnuksilla_2['tutkinnon_osa'] = jaksot_tunnuksilla_2['tutkinnon_osa'].apply(get_tutkinnonosa)
jaksot_tunnuksilla_2['tutkintonimike'] = jaksot_tunnuksilla_2['tutkintonimike'].apply(to_comma_separated_list)
jaksot_tunnuksilla_2['osaamisala'] = jaksot_tunnuksilla_2['osaamisala'].apply(to_comma_separated_list)

jaksot_tunnuksilla_2 = jaksot_tunnuksilla_2[jaksot_tunnuksilla_2['tunnus'] != '']
jaksot_tunnuksilla_2 = jaksot_tunnuksilla_2.drop(columns=['niputuspvm', 'oppija_oid','hoks_id', 'rahoituskausi', 'ohjaaja_nimi','ohjaaja_email', 'ohjaaja_puhelinnumero', 'tutkinnonosa_id', 'tutkinnonosa_tyyppi'])
jaksot_tunnuksilla_2 = jaksot_tunnuksilla_2.set_index('tunnus')
jaksot_tunnuksilla_2.to_csv('jaksot-tunnuksilla-2.csv', sep='|')
jaksot_tunnuksilla_2